# 06_LangGraph_Real_World_Pipelines - Xây dựng Pipeline AI Thực tế

## 🎯 Mục tiêu học tập

Trong notebook này, chúng ta sẽ học cách:
- Hiểu tại sao LangGraph phù hợp cho các pipeline AI phức tạp
- Xây dựng pipeline phân tích và tóm tắt tài liệu (RAG nâng cao)
- Tạo hệ thống hỗ trợ quyết định với nhiều bước tương tác
- Quản lý trạng thái và xử lý lỗi trong pipeline thực tế
- Áp dụng conditional logic và branching phức tạp

## 📋 Tổng quan

### Tại sao LangGraph cho Pipeline AI?

**LangGraph vs Chains đơn giản:**
- **Quản lý trạng thái**: Theo dõi thông tin qua nhiều bước
- **Điều hướng có điều kiện**: Rẽ nhánh dựa trên kết quả trung gian
- **Khả năng mở rộng**: Dễ dàng thêm/bớt các bước xử lý
- **Debugging**: Theo dõi và kiểm soát từng bước thực thi
- **Error handling**: Xử lý lỗi và retry logic tốt hơn

## 🛠️ Cài đặt & Cấu hình

In [ ]:
# Kiểm tra và cài đặt các thư viện cần thiết
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} đã được cài đặt")
    except ImportError:
        print(f"⚠️ Đang cài đặt {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Cài đặt các thư viện cần thiết
packages = [
    "langgraph",
    "langchain-anthropic", 
    "langchain-community",
    "langchain-core",
    "requests",
    "beautifulsoup4",
    "python-dotenv"
]

for package in packages:
    install_package(package)

In [ ]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Kiểm tra API key
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
if not ANTHROPIC_API_KEY:
    print("⚠️ Vui lòng đặt ANTHROPIC_API_KEY trong file .env")
    ANTHROPIC_API_KEY = input("Nhập ANTHROPIC_API_KEY: ")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
else:
    print("✅ ANTHROPIC_API_KEY đã được cấu hình")

In [ ]:
# Import các thư viện cần thiết
from typing import TypedDict, List, Dict, Any, Optional, Annotated
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime

print("✅ Đã import thành công tất cả thư viện cần thiết")

## 🔧 Setup LLM và Utilities

In [ ]:
# Khởi tạo LLM
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.1,
    max_tokens=4000
)

print("✅ Đã khởi tạo ChatAnthropic")

# Test kết nối
try:
    test_response = llm.invoke([HumanMessage(content="Xin chào, bạn có thể trả lời bằng tiếng Việt không?")])
    print(f"🔗 Kết nối thành công: {test_response.content[:100]}...")
except Exception as e:
    print(f"❌ Lỗi kết nối: {e}")

In [ ]:
# Utility functions cho document processing
def extract_text_from_url(url: str) -> str:
    """Trích xuất văn bản từ URL"""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Loại bỏ script và style elements
        for script in soup(["script", "style"]):
            script.extract()
        
        # Lấy text
        text = soup.get_text()
        
        # Làm sạch text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text[:5000]  # Giới hạn để tránh quá dài
    except Exception as e:
        return f"Lỗi khi trích xuất từ {url}: {str(e)}"

def chunk_text(text: str, chunk_size: int = 1000, overlap: int = 200) -> List[str]:
    """Chia text thành các đoạn nhỏ với overlap"""
    if len(text) <= chunk_size:
        return [text]
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        
        # Tìm điểm ngắt tự nhiên (dấu câu hoặc khoảng trắng)
        if end < len(text):
            for i in range(end, max(start + chunk_size - 200, start), -1):
                if text[i] in '.!?\n':
                    end = i + 1
                    break
                elif text[i] == ' ':
                    end = i
                    break
        
        chunks.append(text[start:end].strip())
        start = end - overlap
        
        if start >= len(text):
            break
    
    return chunks

print("✅ Đã định nghĩa utility functions")

## 📊 Pipeline 1: Phân tích & Tóm tắt Tài liệu (RAG Nâng cao)

Pipeline này sẽ:
1. **Tải tài liệu** từ URL hoặc text
2. **Chia đoạn** thông minh
3. **Truy xuất thông tin** liên quan
4. **Đánh giá độ liên quan** của thông tin
5. **Tóm tắt** hoặc **trả lời câu hỏi** dựa trên thông tin đã lọc
6. **Xử lý feedback** và cải thiện kết quả

In [ ]:
# Định nghĩa State cho Document Analysis Pipeline
class DocumentAnalysisState(TypedDict):
    # Input
    query: str
    document_source: str  # URL hoặc text
    
    # Intermediate states
    raw_content: str
    chunks: List[str]
    relevant_chunks: List[str]
    relevance_scores: List[float]
    
    # Output
    analysis_result: str
    confidence_score: float
    suggestions: List[str]
    
    # Control flow
    needs_refinement: bool
    error_message: Optional[str]
    step_count: int

print("✅ Đã định nghĩa DocumentAnalysisState")

In [ ]:
# Node functions cho Document Analysis Pipeline

def load_document(state: DocumentAnalysisState) -> DocumentAnalysisState:
    """Tải tài liệu từ nguồn"""
    print(f"📄 Đang tải tài liệu từ: {state['document_source'][:100]}...")
    
    try:
        if state['document_source'].startswith(('http://', 'https://')):
            content = extract_text_from_url(state['document_source'])
        else:
            content = state['document_source']
        
        return {
            **state,
            'raw_content': content,
            'error_message': None,
            'step_count': state.get('step_count', 0) + 1
        }
    except Exception as e:
        return {
            **state,
            'error_message': f"Lỗi tải tài liệu: {str(e)}",
            'step_count': state.get('step_count', 0) + 1
        }

def chunk_document(state: DocumentAnalysisState) -> DocumentAnalysisState:
    """Chia tài liệu thành các đoạn nhỏ"""
    print("✂️ Đang chia tài liệu thành các đoạn...")
    
    if state.get('error_message'):
        return state
    
    chunks = chunk_text(state['raw_content'])
    print(f"📋 Đã tạo {len(chunks)} đoạn văn bản")
    
    return {
        **state,
        'chunks': chunks,
        'step_count': state['step_count'] + 1
    }

def retrieve_relevant_chunks(state: DocumentAnalysisState) -> DocumentAnalysisState:
    """Truy xuất các đoạn liên quan đến query"""
    print(f"🔍 Đang tìm kiếm thông tin liên quan đến: '{state['query']}'")
    
    if state.get('error_message'):
        return state
    
    # Sử dụng LLM để đánh giá mức độ liên quan
    relevance_prompt = ChatPromptTemplate.from_template(
        """Bạn là một chuyên gia đánh giá mức độ liên quan của thông tin.
        
Query: {query}
Đoạn văn bản: {chunk}

Hãy đánh giá mức độ liên quan của đoạn văn bản này với query trên thang điểm từ 0 đến 1.
Chỉ trả về một số thập phân từ 0.0 đến 1.0, không giải thích thêm.
Ví dụ: 0.8"""
    )
    
    relevant_chunks = []
    relevance_scores = []
    
    for chunk in state['chunks']:
        try:
            score_response = llm.invoke(
                relevance_prompt.format_messages(query=state['query'], chunk=chunk[:500])
            )
            
            # Trích xuất điểm số
            score_text = score_response.content.strip()
            score = float(re.findall(r'\d+\.\d+|\d+', score_text)[0])
            
            if score > 0.3:  # Ngưỡng liên quan
                relevant_chunks.append(chunk)
                relevance_scores.append(score)
                
        except (ValueError, IndexError):
            # Nếu không thể parse điểm số, bỏ qua chunk này
            continue
    
    # Sắp xếp theo điểm số giảm dần
    sorted_pairs = sorted(zip(relevant_chunks, relevance_scores), 
                         key=lambda x: x[1], reverse=True)
    
    if sorted_pairs:
        relevant_chunks, relevance_scores = zip(*sorted_pairs)
        relevant_chunks = list(relevant_chunks[:5])  # Top 5 chunks
        relevance_scores = list(relevance_scores[:5])
    else:
        relevant_chunks = []
        relevance_scores = []
    
    print(f"✅ Tìm được {len(relevant_chunks)} đoạn liên quan")
    
    return {
        **state,
        'relevant_chunks': relevant_chunks,
        'relevance_scores': relevance_scores,
        'step_count': state['step_count'] + 1
    }

def analyze_and_summarize(state: DocumentAnalysisState) -> DocumentAnalysisState:
    """Phân tích và tóm tắt thông tin"""
    print("🧠 Đang phân tích và tóm tắt thông tin...")
    
    if state.get('error_message'):
        return state
    
    if not state['relevant_chunks']:
        return {
            **state,
            'analysis_result': "Không tìm thấy thông tin liên quan đến câu hỏi của bạn trong tài liệu.",
            'confidence_score': 0.0,
            'suggestions': ["Thử với câu hỏi khác", "Kiểm tra lại tài liệu nguồn"],
            'needs_refinement': True,
            'step_count': state['step_count'] + 1
        }
    
    # Tạo context từ các chunks liên quan
    context = "\n\n".join([
        f"Đoạn {i+1} (độ liên quan: {score:.2f}):\n{chunk}"
        for i, (chunk, score) in enumerate(zip(state['relevant_chunks'], state['relevance_scores']))
    ])
    
    analysis_prompt = ChatPromptTemplate.from_template(
        """Bạn là một chuyên gia phân tích tài liệu. Hãy phân tích và trả lời câu hỏi dựa trên thông tin được cung cấp.

Câu hỏi: {query}

Thông tin từ tài liệu:
{context}

Hãy:
1. Trả lời câu hỏi một cách chi tiết và chính xác
2. Trích dẫn thông tin từ tài liệu
3. Đánh giá độ tin cậy của câu trả lời (0-1)
4. Đưa ra 2-3 gợi ý để tìm hiểu thêm

Định dạng trả lời:
**Câu trả lời:**
[Câu trả lời chi tiết]

**Độ tin cậy:** [0.0-1.0]

**Gợi ý thêm:**
- [Gợi ý 1]
- [Gợi ý 2]
- [Gợi ý 3]"""
    )
    
    try:
        response = llm.invoke(
            analysis_prompt.format_messages(query=state['query'], context=context)
        )
        
        result = response.content
        
        # Trích xuất confidence score
        confidence_match = re.search(r'\*\*Độ tin cậy:\*\*\s*([0-9.]+)', result)
        confidence_score = float(confidence_match.group(1)) if confidence_match else 0.5
        
        # Trích xuất suggestions
        suggestions_section = re.search(r'\*\*Gợi ý thêm:\*\*\s*(.+)', result, re.DOTALL)
        suggestions = []
        if suggestions_section:
            suggestion_lines = suggestions_section.group(1).strip().split('\n')
            suggestions = [line.strip('- ').strip() for line in suggestion_lines if line.strip().startswith('-')]
        
        return {
            **state,
            'analysis_result': result,
            'confidence_score': confidence_score,
            'suggestions': suggestions,
            'needs_refinement': confidence_score < 0.6,
            'step_count': state['step_count'] + 1
        }
        
    except Exception as e:
        return {
            **state,
            'error_message': f"Lỗi phân tích: {str(e)}",
            'step_count': state['step_count'] + 1
        }

print("✅ Đã định nghĩa các node functions cho Document Analysis")

In [ ]:
# Tạo Document Analysis Graph
def create_document_analysis_graph():
    """Tạo workflow graph cho phân tích tài liệu"""
    
    workflow = StateGraph(DocumentAnalysisState)
    
    # Thêm các nodes
    workflow.add_node("load_document", load_document)
    workflow.add_node("chunk_document", chunk_document)
    workflow.add_node("retrieve_relevant", retrieve_relevant_chunks)
    workflow.add_node("analyze_summarize", analyze_and_summarize)
    
    # Định nghĩa flow
    workflow.add_edge(START, "load_document")
    
    # Conditional edge sau load_document
    def should_continue_after_load(state: DocumentAnalysisState) -> str:
        if state.get('error_message'):
            return END
        return "chunk_document"
    
    workflow.add_conditional_edges(
        "load_document",
        should_continue_after_load,
        {"chunk_document": "chunk_document", END: END}
    )
    
    workflow.add_edge("chunk_document", "retrieve_relevant")
    workflow.add_edge("retrieve_relevant", "analyze_summarize")
    
    # Conditional edge sau analyze_summarize
    def should_refine(state: DocumentAnalysisState) -> str:
        if state.get('needs_refinement') and state.get('step_count', 0) < 5:
            return "retrieve_relevant"  # Retry với threshold thấp hơn
        return END
    
    workflow.add_conditional_edges(
        "analyze_summarize",
        should_refine,
        {"retrieve_relevant": "retrieve_relevant", END: END}
    )
    
    # Compile graph
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

# Tạo graph
doc_analysis_app = create_document_analysis_graph()
print("✅ Đã tạo Document Analysis Graph")

In [ ]:
# Test Document Analysis Pipeline
def test_document_analysis():
    """Test pipeline phân tích tài liệu"""
    
    # Sample document
    sample_doc = """
    Trí tuệ nhân tạo (AI) đang thay đổi cách chúng ta làm việc và sống. 
    Machine Learning là một nhánh quan trọng của AI, cho phép máy tính học từ dữ liệu mà không cần lập trình cụ thể.
    
    Deep Learning, một phần của Machine Learning, sử dụng mạng neural nhân tạo để xử lý dữ liệu phức tạp.
    Các ứng dụng của AI bao gồm:
    - Xử lý ngôn ngữ tự nhiên (NLP)
    - Computer Vision
    - Robotics
    - Autonomous vehicles
    
    LangChain là một framework mạnh mẽ để xây dựng ứng dụng AI sử dụng Large Language Models (LLMs).
    LangGraph mở rộng LangChain bằng cách cung cấp khả năng tạo workflow phức tạp với state management.
    
    Trong tương lai, AI sẽ tiếp tục phát triển và tích hợp sâu hơn vào cuộc sống hàng ngày.
    """
    
    # Test cases
    test_cases = [
        {
            "query": "LangGraph là gì và có tác dụng gì?",
            "document_source": sample_doc
        },
        {
            "query": "Các ứng dụng của AI có gì?",
            "document_source": sample_doc
        }
    ]
    
    for i, test_case in enumerate(test_cases):
        print(f"\n{'='*60}")
        print(f"🧪 TEST CASE {i+1}: {test_case['query']}")
        print(f"{'='*60}")
        
        # Tạo initial state
        initial_state = {
            "query": test_case['query'],
            "document_source": test_case['document_source'],
            "step_count": 0,
            "needs_refinement": False
        }
        
        # Chạy pipeline
        config = {"configurable": {"thread_id": f"test_{i}"}}
        
        try:
            # Execute graph
            result = doc_analysis_app.invoke(initial_state, config)
            
            # Display results
            if result.get('error_message'):
                print(f"❌ Lỗi: {result['error_message']}")
            else:
                print(f"📊 Kết quả phân tích:")
                print(f"📈 Độ tin cậy: {result.get('confidence_score', 0):.2f}")
                print(f"📝 Kết quả:\n{result.get('analysis_result', 'Không có kết quả')}")
                
                if result.get('suggestions'):
                    print(f"\n💡 Gợi ý:")
                    for suggestion in result['suggestions']:
                        print(f"   • {suggestion}")
                        
        except Exception as e:
            print(f"❌ Lỗi thực thi: {str(e)}")

# Chạy test
test_document_analysis()

## 🤖 Pipeline 2: Hệ thống Hỗ trợ Quyết định (Decision Support Agent)

Pipeline này sẽ:
1. **Phân tích yêu cầu** của người dùng
2. **Tìm kiếm thông tin** từ nhiều nguồn
3. **Phân tích dữ liệu** và so sánh các lựa chọn
4. **Đề xuất giải pháp** với lý do
5. **Tương tác với người dùng** để làm rõ
6. **Đưa ra quyết định cuối cùng**

In [ ]:
# Định nghĩa State cho Decision Support Pipeline
class DecisionSupportState(TypedDict):
    # Input
    user_request: str
    context: Dict[str, Any]  # Thông tin bổ sung từ user
    
    # Analysis phase
    request_type: str  # "comparison", "recommendation", "analysis", "planning"
    key_factors: List[str]
    information_needs: List[str]
    
    # Research phase
    collected_info: Dict[str, Any]
    sources: List[str]
    
    # Decision phase
    options: List[Dict[str, Any]]
    analysis_results: Dict[str, Any]
    recommendations: List[str]
    
    # Interaction phase
    clarifications_needed: List[str]
    user_feedback: Optional[str]
    
    # Output
    final_decision: str
    reasoning: str
    confidence_level: float
    next_steps: List[str]
    
    # Control
    needs_clarification: bool
    is_complete: bool
    iteration_count: int

print("✅ Đã định nghĩa DecisionSupportState")

In [ ]:
# Node functions cho Decision Support Pipeline

def analyze_request(state: DecisionSupportState) -> DecisionSupportState:
    """Phân tích yêu cầu của người dùng"""
    print(f"🔍 Đang phân tích yêu cầu: {state['user_request'][:100]}...")
    
    analysis_prompt = ChatPromptTemplate.from_template(
        """Bạn là một chuyên gia phân tích yêu cầu. Hãy phân tích yêu cầu sau và trả lời theo định dạng JSON.

Yêu cầu: {request}
Context: {context}

Hãy xác định:
1. Loại yêu cầu (comparison/recommendation/analysis/planning)
2. Các yếu tố quan trọng cần xem xét
3. Thông tin cần thu thập

Trả lời theo định dạng JSON:
{{
    "request_type": "loại yêu cầu",
    "key_factors": ["yếu tố 1", "yếu tố 2", "yếu tố 3"],
    "information_needs": ["thông tin cần 1", "thông tin cần 2"]
}}"""
    )
    
    try:
        response = llm.invoke(
            analysis_prompt.format_messages(
                request=state['user_request'],
                context=json.dumps(state.get('context', {}), ensure_ascii=False)
            )
        )
        
        # Parse JSON response
        result_text = response.content.strip()
        json_match = re.search(r'\{[^}]+\}', result_text, re.DOTALL)
        
        if json_match:
            result_json = json.loads(json_match.group())
            
            return {
                **state,
                'request_type': result_json.get('request_type', 'analysis'),
                'key_factors': result_json.get('key_factors', []),
                'information_needs': result_json.get('information_needs', []),
                'iteration_count': state.get('iteration_count', 0) + 1
            }
        else:
            # Fallback nếu không parse được JSON
            return {
                **state,
                'request_type': 'analysis',
                'key_factors': ['Chất lượng', 'Chi phí', 'Thời gian'],
                'information_needs': ['Thông tin cơ bản', 'So sánh lựa chọn'],
                'iteration_count': state.get('iteration_count', 0) + 1
            }
            
    except Exception as e:
        print(f"⚠️ Lỗi phân tích yêu cầu: {e}")
        return {
            **state,
            'request_type': 'analysis',
            'key_factors': ['Yếu tố chính'],
            'information_needs': ['Thông tin cần thiết'],
            'iteration_count': state.get('iteration_count', 0) + 1
        }

def collect_information(state: DecisionSupportState) -> DecisionSupportState:
    """Thu thập thông tin cần thiết"""
    print(f"📊 Đang thu thập thông tin cho {len(state['information_needs'])} nhu cầu")
    
    # Mô phỏng việc thu thập thông tin từ nhiều nguồn
    collected_info = {}
    sources = []
    
    research_prompt = ChatPromptTemplate.from_template(
        """Bạn là một nhà nghiên cứu chuyên nghiệp. Hãy cung cấp thông tin chi tiết về:

Chủ đề nghiên cứu: {topic}
Yêu cầu gốc: {original_request}
Yếu tố quan trọng: {key_factors}

Hãy cung cấp:
1. Thông tin chi tiết và cập nhật
2. Ưu điểm và nhược điểm
3. Các lựa chọn có thể
4. Đánh giá dựa trên các yếu tố quan trọng

Định dạng trả lời:
**Thông tin tổng quan:**
[Thông tin chi tiết]

**Các lựa chọn:**
1. [Lựa chọn 1]: [Mô tả và đánh giá]
2. [Lựa chọn 2]: [Mô tả và đánh giá]

**Phân tích theo yếu tố:**
[Phân tích dựa trên key_factors]"""
    )
    
    for info_need in state['information_needs']:
        try:
            response = llm.invoke(
                research_prompt.format_messages(
                    topic=info_need,
                    original_request=state['user_request'],
                    key_factors=', '.join(state['key_factors'])
                )
            )
            
            collected_info[info_need] = response.content
            sources.append(f"Nghiên cứu nội bộ - {info_need}")
            
        except Exception as e:
            print(f"⚠️ Lỗi thu thập thông tin cho {info_need}: {e}")
            collected_info[info_need] = f"Không thể thu thập thông tin cho {info_need}"
    
    print(f"✅ Đã thu thập thông tin từ {len(sources)} nguồn")
    
    return {
        **state,
        'collected_info': collected_info,
        'sources': sources,
        'iteration_count': state['iteration_count'] + 1
    }

def analyze_and_recommend(state: DecisionSupportState) -> DecisionSupportState:
    """Phân tích thông tin và đưa ra khuyến nghị"""
    print("🧠 Đang phân tích thông tin và tạo khuyến nghị...")
    
    # Tổng hợp thông tin
    all_info = "\n\n".join([
        f"**{topic}:**\n{info}"
        for topic, info in state['collected_info'].items()
    ])
    
    recommendation_prompt = ChatPromptTemplate.from_template(
        """Bạn là một chuyên gia tư vấn quyết định. Dựa trên thông tin đã thu thập, hãy đưa ra phân tích và khuyến nghị.

Yêu cầu gốc: {request}
Loại yêu cầu: {request_type}
Yếu tố quan trọng: {key_factors}

Thông tin đã thu thập:
{collected_info}

Hãy cung cấp:
1. Phân tích chi tiết các lựa chọn
2. So sánh ưu nhược điểm
3. Khuyến nghị cụ thể với lý do
4. Đánh giá độ tin cậy (0-1)
5. Các câu hỏi cần làm rõ thêm (nếu có)

Định dạng:
**PHÂN TÍCH:**
[Phân tích chi tiết]

**KHUYẾN NGHỊ:**
[Khuyến nghị cụ thể]

**LÝ DO:**
[Lý do cho khuyến nghị]

**ĐỘ TIN CẬY:** [0.0-1.0]

**CÂU HỎI LÀM RÕ:**
- [Câu hỏi 1]
- [Câu hỏi 2]"""
    )
    
    try:
        response = llm.invoke(
            recommendation_prompt.format_messages(
                request=state['user_request'],
                request_type=state['request_type'],
                key_factors=', '.join(state['key_factors']),
                collected_info=all_info
            )
        )
        
        result = response.content
        
        # Trích xuất confidence level
        confidence_match = re.search(r'\*\*ĐỘ TIN CẬY:\*\*\s*([0-9.]+)', result)
        confidence_level = float(confidence_match.group(1)) if confidence_match else 0.7
        
        # Trích xuất clarifications
        clarification_section = re.search(r'\*\*CÂU HỎI LÀM RÕ:\*\*\s*(.+)', result, re.DOTALL)
        clarifications = []
        if clarification_section:
            clarification_lines = clarification_section.group(1).strip().split('\n')
            clarifications = [line.strip('- ').strip() for line in clarification_lines if line.strip().startswith('-')]
        
        return {
            **state,
            'analysis_results': {'full_analysis': result},
            'confidence_level': confidence_level,
            'clarifications_needed': clarifications,
            'needs_clarification': len(clarifications) > 0 and confidence_level < 0.8,
            'iteration_count': state['iteration_count'] + 1
        }
        
    except Exception as e:
        print(f"⚠️ Lỗi phân tích: {e}")
        return {
            **state,
            'analysis_results': {'error': str(e)},
            'confidence_level': 0.3,
            'needs_clarification': True,
            'iteration_count': state['iteration_count'] + 1
        }

def finalize_decision(state: DecisionSupportState) -> DecisionSupportState:
    """Hoàn thiện quyết định cuối cùng"""
    print("✅ Đang hoàn thiện quyết định cuối cùng...")
    
    final_prompt = ChatPromptTemplate.from_template(
        """Dựa trên phân tích đã thực hiện, hãy đưa ra quyết định cuối cùng và kế hoạch hành động.

Yêu cầu gốc: {request}
Phân tích đã thực hiện: {analysis}
Độ tin cậy: {confidence}

Hãy cung cấp:
1. Quyết định cuối cùng rõ ràng
2. Lý do chi tiết
3. Các bước tiếp theo cần thực hiện

Định dạng:
**QUYẾT ĐỊNH CUỐI CÙNG:**
[Quyết định cụ thể]

**LÝ DO:**
[Lý do chi tiết]

**CÁC BƯỚC TIẾP THEO:**
1. [Bước 1]
2. [Bước 2]
3. [Bước 3]"""
    )
    
    try:
        response = llm.invoke(
            final_prompt.format_messages(
                request=state['user_request'],
                analysis=state['analysis_results'].get('full_analysis', 'Không có phân tích'),
                confidence=state['confidence_level']
            )
        )
        
        result = response.content
        
        # Trích xuất next steps
        steps_section = re.search(r'\*\*CÁC BƯỚC TIẾP THEO:\*\*\s*(.+)', result, re.DOTALL)
        next_steps = []
        if steps_section:
            step_lines = steps_section.group(1).strip().split('\n')
            next_steps = [re.sub(r'^\d+\.\s*', '', line.strip()) for line in step_lines if re.match(r'^\d+\.', line.strip())]
        
        return {
            **state,
            'final_decision': result,
            'reasoning': result,
            'next_steps': next_steps,
            'is_complete': True,
            'iteration_count': state['iteration_count'] + 1
        }
        
    except Exception as e:
        return {
            **state,
            'final_decision': f"Lỗi hoàn thiện quyết định: {str(e)}",
            'is_complete': True,
            'iteration_count': state['iteration_count'] + 1
        }

print("✅ Đã định nghĩa các node functions cho Decision Support")

In [ ]:
# Tạo Decision Support Graph
def create_decision_support_graph():
    """Tạo workflow graph cho hỗ trợ quyết định"""
    
    workflow = StateGraph(DecisionSupportState)
    
    # Thêm các nodes
    workflow.add_node("analyze_request", analyze_request)
    workflow.add_node("collect_info", collect_information)
    workflow.add_node("analyze_recommend", analyze_and_recommend)
    workflow.add_node("finalize", finalize_decision)
    
    # Định nghĩa flow
    workflow.add_edge(START, "analyze_request")
    workflow.add_edge("analyze_request", "collect_info")
    workflow.add_edge("collect_info", "analyze_recommend")
    
    # Conditional edge sau analyze_recommend
    def should_clarify(state: DecisionSupportState) -> str:
        if (state.get('needs_clarification') and 
            state.get('iteration_count', 0) < 3):
            return "collect_info"  # Collect more info
        return "finalize"
    
    workflow.add_conditional_edges(
        "analyze_recommend",
        should_clarify,
        {"collect_info": "collect_info", "finalize": "finalize"}
    )
    
    workflow.add_edge("finalize", END)
    
    # Compile graph
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

# Tạo graph
decision_support_app = create_decision_support_graph()
print("✅ Đã tạo Decision Support Graph")

In [ ]:
# Test Decision Support Pipeline
def test_decision_support():
    """Test pipeline hỗ trợ quyết định"""
    
    test_cases = [
        {
            "user_request": "Tôi muốn học một ngôn ngữ lập trình mới để phát triển ứng dụng AI. Nên chọn Python hay JavaScript?",
            "context": {
                "current_skills": ["HTML", "CSS", "basic programming"],
                "goal": "AI application development",
                "time_available": "3-6 months"
            }
        },
        {
            "user_request": "Công ty tôi cần chọn giữa việc xây dựng chatbot nội bộ hay thuê dịch vụ bên ngoài. Giúp tôi quyết định.",
            "context": {
                "company_size": "50-100 employees",
                "budget": "moderate",
                "technical_team": "limited"
            }
        }
    ]
    
    for i, test_case in enumerate(test_cases):
        print(f"\n{'='*80}")
        print(f"🤖 DECISION SUPPORT TEST {i+1}")
        print(f"📋 Yêu cầu: {test_case['user_request']}")
        print(f"{'='*80}")
        
        # Tạo initial state
        initial_state = {
            "user_request": test_case['user_request'],
            "context": test_case['context'],
            "iteration_count": 0,
            "needs_clarification": False,
            "is_complete": False
        }
        
        # Chạy pipeline
        config = {"configurable": {"thread_id": f"decision_test_{i}"}}
        
        try:
            # Execute graph
            result = decision_support_app.invoke(initial_state, config)
            
            # Display results
            print(f"\n📊 KẾT QUẢ PHÂN TÍCH:")
            print(f"🎯 Loại yêu cầu: {result.get('request_type', 'N/A')}")
            print(f"🔑 Yếu tố quan trọng: {', '.join(result.get('key_factors', []))}")
            print(f"📈 Độ tin cậy: {result.get('confidence_level', 0):.2f}")
            
            if result.get('final_decision'):
                print(f"\n✅ QUYẾT ĐỊNH CUỐI CÙNG:")
                print(result['final_decision'])
            
            if result.get('next_steps'):
                print(f"\n📋 CÁC BƯỚC TIẾP THEO:")
                for j, step in enumerate(result['next_steps'], 1):
                    print(f"   {j}. {step}")
            
            if result.get('clarifications_needed'):
                print(f"\n❓ CÂU HỎI CẦN LÀM RÕ:")
                for clarification in result['clarifications_needed']:
                    print(f"   • {clarification}")
                    
        except Exception as e:
            print(f"❌ Lỗi thực thi: {str(e)}")

# Chạy test
test_decision_support()

## 📈 Phân tích Cấu trúc và Hiệu suất Pipeline

### 🔄 Cấu trúc Graph

Hãy phân tích cấu trúc của các pipeline chúng ta đã tạo:

In [ ]:
# Phân tích cấu trúc Graph
def analyze_graph_structure():
    """Phân tích và hiển thị cấu trúc của các graph"""
    
    print("📊 PHÂN TÍCH CẤU TRÚC PIPELINE\n")
    
    # Document Analysis Pipeline
    print("🔍 DOCUMENT ANALYSIS PIPELINE:")
    print("├── START")
    print("├── load_document (Tải tài liệu)")
    print("│   ├── Success → chunk_document")
    print("│   └── Error → END")
    print("├── chunk_document (Chia đoạn)")
    print("├── retrieve_relevant (Truy xuất liên quan)")
    print("├── analyze_summarize (Phân tích & tóm tắt)")
    print("│   ├── needs_refinement → retrieve_relevant (retry)")
    print("│   └── complete → END")
    print("└── END\n")
    
    # Decision Support Pipeline
    print("🤖 DECISION SUPPORT PIPELINE:")
    print("├── START")
    print("├── analyze_request (Phân tích yêu cầu)")
    print("├── collect_info (Thu thập thông tin)")
    print("├── analyze_recommend (Phân tích & khuyến nghị)")
    print("│   ├── needs_clarification → collect_info (retry)")
    print("│   └── ready → finalize")
    print("├── finalize (Hoàn thiện quyết định)")
    print("└── END\n")
    
    # Key Features
    print("🔑 TÍNH NĂNG QUAN TRỌNG:")
    print("• State Management: Theo dõi trạng thái qua các bước")
    print("• Conditional Routing: Rẽ nhánh dựa trên kết quả")
    print("• Error Handling: Xử lý lỗi và retry logic")
    print("• Memory Persistence: Lưu trạng thái với MemorySaver")
    print("• Iterative Refinement: Cải thiện kết quả qua nhiều lần thực hiện")
    
    # Benefits
    print("\n✅ LỢI ÍCH CỦA LANGGRAPH:")
    print("• Modularity: Dễ dàng thêm/bớt/thay đổi các bước")
    print("• Debugging: Theo dõi từng bước thực thi")
    print("• Scalability: Mở rộng pipeline phức tạp")
    print("• Reliability: Xử lý lỗi và retry tốt hơn")
    print("• Flexibility: Thay đổi luồng dựa trên điều kiện")

analyze_graph_structure()

## 🔧 Best Practices và Tối ưu hóa

### Các nguyên tắc quan trọng khi xây dựng pipeline thực tế:

In [ ]:
# Best Practices cho LangGraph Pipelines
def demonstrate_best_practices():
    """Minh họa các best practices"""
    
    print("🏆 BEST PRACTICES CHO LANGGRAPH PIPELINES\n")
    
    print("1. 📋 STATE DESIGN:")
    print("   • Sử dụng TypedDict cho type safety")
    print("   • Bao gồm control fields (error_message, step_count)")
    print("   • Thiết kế state tối thiểu nhưng đầy đủ")
    print("   • Sử dụng Optional cho các field không bắt buộc\n")
    
    print("2. 🔄 ERROR HANDLING:")
    print("   • Luôn có try-catch trong các node functions")
    print("   • Truyền error thông qua state")
    print("   • Conditional routing dựa trên error status")
    print("   • Graceful degradation khi gặp lỗi\n")
    
    print("3. 🎯 CONDITIONAL LOGIC:")
    print("   • Sử dụng conditional edges cho branching")
    print("   • Đặt điều kiện dựa trên business logic")
    print("   • Tránh vòng lặp vô hạn với iteration counters")
    print("   • Default paths cho các trường hợp edge cases\n")
    
    print("4. 🚀 PERFORMANCE:")
    print("   • Sử dụng MemorySaver cho persistent state")
    print("   • Batch processing khi có thể")
    print("   • Caching kết quả trung gian")
    print("   • Parallel execution cho independent tasks\n")
    
    print("5. 🧪 TESTING:")
    print("   • Test từng node function riêng biệt")
    print("   • Integration tests cho toàn bộ pipeline")
    print("   • Mock external dependencies")
    print("   • Test các edge cases và error conditions\n")
    
    print("6. 📊 MONITORING:")
    print("   • Log state changes tại mỗi node")
    print("   • Track execution time và performance")
    print("   • Monitor success/failure rates")
    print("   • Alert trên các lỗi quan trọng")

demonstrate_best_practices()

## 🔍 Debugging và Monitoring

### Cách debug và monitor pipeline hiệu quả:

In [ ]:
# Debugging utilities
def create_debug_pipeline():
    """Tạo pipeline với debugging capabilities"""
    
    class DebugState(TypedDict):
        input_data: str
        processed_data: str
        result: str
        debug_info: Dict[str, Any]
        execution_log: List[str]
    
    def debug_node_1(state: DebugState) -> DebugState:
        """Node với debugging"""
        start_time = datetime.now()
        
        try:
            # Actual processing
            processed = f"Processed: {state['input_data']}"
            
            # Debug info
            end_time = datetime.now()
            execution_time = (end_time - start_time).total_seconds()
            
            debug_info = state.get('debug_info', {})
            debug_info['node_1'] = {
                'execution_time': execution_time,
                'input_length': len(state['input_data']),
                'output_length': len(processed),
                'timestamp': start_time.isoformat()
            }
            
            execution_log = state.get('execution_log', [])
            execution_log.append(f"Node 1 executed in {execution_time:.3f}s")
            
            return {
                **state,
                'processed_data': processed,
                'debug_info': debug_info,
                'execution_log': execution_log
            }
            
        except Exception as e:
            execution_log = state.get('execution_log', [])
            execution_log.append(f"Node 1 failed: {str(e)}")
            
            return {
                **state,
                'debug_info': state.get('debug_info', {}),
                'execution_log': execution_log
            }
    
    def debug_node_2(state: DebugState) -> DebugState:
        """Node 2 với debugging"""
        start_time = datetime.now()
        
        result = f"Final: {state['processed_data']}"
        
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()
        
        debug_info = state.get('debug_info', {})
        debug_info['node_2'] = {
            'execution_time': execution_time,
            'timestamp': start_time.isoformat()
        }
        
        execution_log = state.get('execution_log', [])
        execution_log.append(f"Node 2 executed in {execution_time:.3f}s")
        
        return {
            **state,
            'result': result,
            'debug_info': debug_info,
            'execution_log': execution_log
        }
    
    # Create graph
    workflow = StateGraph(DebugState)
    workflow.add_node("node_1", debug_node_1)
    workflow.add_node("node_2", debug_node_2)
    
    workflow.add_edge(START, "node_1")
    workflow.add_edge("node_1", "node_2")
    workflow.add_edge("node_2", END)
    
    return workflow.compile()

# Test debug pipeline
def test_debug_pipeline():
    """Test debug pipeline"""
    print("🔍 TESTING DEBUG PIPELINE\n")
    
    debug_app = create_debug_pipeline()
    
    initial_state = {
        "input_data": "Test data for debugging",
        "debug_info": {},
        "execution_log": []
    }
    
    result = debug_app.invoke(initial_state)
    
    print("📊 DEBUG INFORMATION:")
    print(f"Input: {result['input_data']}")
    print(f"Result: {result['result']}")
    
    print("\n⏱️ EXECUTION LOG:")
    for log_entry in result['execution_log']:
        print(f"  • {log_entry}")
    
    print("\n🔧 DEBUG INFO:")
    for node, info in result['debug_info'].items():
        print(f"  {node}:")
        for key, value in info.items():
            print(f"    - {key}: {value}")

test_debug_pipeline()

## 📚 Tài liệu Tham khảo

### Các nguồn tài liệu quan trọng để tìm hiểu thêm:

1. **LangGraph Official Documentation**
   - [LangGraph Introduction](https://langchain-ai.github.io/langgraph/)
   - [How-to Guides](https://langchain-ai.github.io/langgraph/how-tos/)
   - [API Reference](https://langchain-ai.github.io/langgraph/reference/)

2. **Advanced Patterns**
   - [Conditional Edges](https://langchain-ai.github.io/langgraph/how-tos/branching/)
   - [State Management](https://langchain-ai.github.io/langgraph/how-tos/state-model/)
   - [Error Handling](https://langchain-ai.github.io/langgraph/how-tos/error-handling/)

3. **Real-world Examples**
   - [Multi-agent Systems](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/)
   - [RAG Applications](https://langchain-ai.github.io/langgraph/tutorials/rag/)
   - [Chatbot Implementation](https://langchain-ai.github.io/langgraph/tutorials/chatbots/)

4. **Integration Guides**
   - [LangSmith Integration](https://langchain-ai.github.io/langgraph/how-tos/langsmith/)
   - [Anthropic Models](https://python.langchain.com/docs/integrations/chat/anthropic/)
   - [Memory and Persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/)

## 🎯 Kết luận và Bước tiếp theo

### 📝 Tóm tắt những gì đã học:

1. **Pipeline Architecture**: Cách thiết kế và xây dựng pipeline AI có cấu trúc
2. **State Management**: Quản lý trạng thái phức tạp qua nhiều bước
3. **Conditional Logic**: Rẽ nhánh và điều hướng dựa trên kết quả
4. **Error Handling**: Xử lý lỗi và retry logic hiệu quả
5. **Real-world Applications**: Ứng dụng thực tế với RAG và Decision Support

### 🚀 Bước tiếp theo:

1. **Thực hành**: Tự tạo pipeline cho use case riêng của bạn
2. **Tối ưu hóa**: Áp dụng các best practices và monitoring
3. **Mở rộng**: Thêm các tính năng như async processing, parallel execution
4. **Production**: Deploy pipeline với proper infrastructure
5. **Integration**: Tích hợp với các hệ thống khác trong organization

### 💡 Gợi ý cho dự án tiếp theo:

- **Content Generation Pipeline**: Multi-step content creation và review
- **Data Analysis Pipeline**: Automated data processing và insights
- **Customer Service Agent**: Intelligent routing và response system
- **Research Assistant**: Automated research và report generation
- **Code Review Pipeline**: Automated code analysis và suggestions

In [ ]:
# Final message
print("🎉 HOÀN THÀNH NOTEBOOK 06 - LANGGRAPH REAL WORLD PIPELINES")
print("\n" + "="*60)
print("✅ Bạn đã học được:")
print("   • Xây dựng pipeline RAG nâng cao")
print("   • Tạo hệ thống hỗ trợ quyết định")
print("   • Quản lý state và conditional logic")
print("   • Best practices và debugging")
print("\n🚀 Hãy tiếp tục khám phá và xây dựng các pipeline phức tạp hơn!")
print("📚 Tham khảo tài liệu chính thức để tìm hiểu thêm các tính năng nâng cao.")
print("="*60)